Using sklearn, 
- [ ] apply a stratified 70-30 training-testing split with a fixed seed (random_state=1), 
- [ ] assess in a single plot the training and testing accuracies of a decision tree with no depth limits (and remaining default behavior) 
- [ ] for a varying number of selected features in {5,10,40,100,250,700}. 
- [ ] Feature selection should be performed before decision tree learning considering the discriminative power of the input variables according to mutual information criterion (mutual_info_classif).

In [22]:
import matplotlib.pyplot as plt
from sklearn import metrics, datasets, tree
from sklearn.model_selection import train_test_split
from scipy.io.arff import loadarff
import pandas as pd

data = loadarff('pd_speech.arff')
df = pd.DataFrame(data[0])
df = df.dropna()
df['class'] = df['class'].str.decode('utf-8')

df.head()

X=df.drop('class', axis=1)
y = df['class']

In [23]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, stratify=y)
print("#training obs=", len(X_train), "\ntesting obs=", len(X_test))


#acho q e suposto o que ta aqui para baixo estar num loop but idk
predictor = tree.DecisionTreeClassifier(random_state=1, max_features=n)
predictor.fit(X_train, y_train)

y_pred = predictor.predict(X_test)
figure = plt.figure(figsize=(200, 100))
tree.plot_tree(predictor, feature_names = df.columns, class_names=df['class'].unique(), impurity=False)
plt.show()

print("\nPredictor accuracy:", round(metrics.accuracy_score(y_test, y_pred), 2))

#training obs= 529 
testing obs= 227



Predictor accuracy: 0.84
